In [18]:
import numpy as np
import pandas as pd
import obspy
from obspy.taup import TauPyModel
from scipy.stats import norm
import scipy.interpolate as spi
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [19]:
df_mars = pd.read_csv("Downloads/modified_mars_seismic_data_m_005.csv")


mars_velocity_model = {
    "depth_km": [0, 400, 800, 1200, 1600, 1800, 2000, 2200, 2500, 2700, 2900, 3200],
    "p_wave_velocity_km_s": [5.0, 6.0, 7.5, 8.2, 8.8, 9.2, 9.5, 9.7, 10.0, 10.2, 10.4, 10.5],
    "s_wave_velocity_km_s": [3.0, 3.5, 4.2, 4.6, 4.9, 5.1, 5.3, 5.4, 5.5, 5.6, 5.6, 0.0],
    "density_g_cm3": [3.5, 3.6, 3.7, 3.8, 4.0, 4.2, 4.5, 4.7, 5.0, 5.2, 5.4, 5.6]
}


mars_depths = np.array(mars_velocity_model["depth_km"])
mars_p_velocity = np.array(mars_velocity_model["p_wave_velocity_km_s"])
mars_s_velocity = np.array(mars_velocity_model["s_wave_velocity_km_s"])
mars_density = np.array(mars_velocity_model["density_g_cm3"])
mars_p_interp = spi.interp1d(mars_depths, mars_p_velocity, kind="cubic", fill_value="extrapolate")
mars_s_interp = spi.interp1d(mars_depths, mars_s_velocity, kind="cubic", fill_value="extrapolate")
mars_density_interp = spi.interp1d(mars_depths, mars_density, kind="cubic", fill_value="extrapolate")

<div style="background-color:#121212; color:#E0E0E0; padding:20px; border-radius:8px;font-family: 'Dancing Script', cursive;">
    <h2 style="color:#F1C40F;">Defining  advanced seismic likelihood function</h2>
    <p></p>
</div>

In [20]:
def seismic_likelihood_advanced(depth, core_radius, sigma=0.1):
    
    expected_s_wave = mars_s_interp(depth)
    expected_density = mars_density_interp(depth)
    
    if expected_s_wave <= 0.1 and expected_density > 5.0:
        return np.exp(-((expected_density / core_radius) ** 2) / (2 * sigma**2))
    
    return 0.1


<div style="background-color:#121212; color:#E0E0E0; padding:20px; border-radius:8px;font-family: 'Dancing Script', cursive;">
    <h2 style="color:#F1C40F;">Computing posterior probabilities and final core radius using Bayesian Inference</h2>
    <p></p>
</div>

In [21]:
depth_samples = np.linspace(1000, 2500, 100)
posterior_probs_advanced = np.array([seismic_likelihood_advanced(d, 1700) for d in depth_samples])


bayesian_core_radius_advanced = depth_samples[np.argmax(posterior_probs_advanced)]


likelihoods = norm.pdf(depth_samples, loc=bayesian_core_radius_advanced, scale=100)


posterior_final = posterior_probs_advanced * likelihoods
posterior_final /= posterior_final.sum()  



<div style="background-color:#121212; color:#E0E0E0; padding:20px; border-radius:8px;font-family: 'Dancing Script', cursive;">
    <h2 style="color:#F1C40F;">Final core radius estimation using Bayesian results</h2>
    <p></p>
</div>

In [22]:
final_core_radius_bayesian_advanced = depth_samples[np.argmax(posterior_final)]


mars_radius = 3389.5  
mars_core_radius_final = mars_radius - final_core_radius_bayesian_advanced

print(f"Estimated Core Radius of Mars (Bayesian Inference): {mars_core_radius_final:.2f} km")


mars_core_radius_label = np.full(len(df_mars), mars_core_radius_final)  

features = [
     "P-wave Velocity (km/s)", "S-wave Velocity (km/s)",
    "Density (g/cm^3)"
]

Estimated Core Radius of Mars (Bayesian Inference): 2389.50 km



 

<div style="background-color:#121212; color:#E0E0E0; padding:20px; border-radius:8px;font-family: 'Dancing Script', cursive;">
    <h2 style="color:#F1C40F;"> Training using Gradient Boosting Regressor and Deep Learning</h2>
    <p></p>
</div>

In [23]:
X = df_mars[features]
y = mars_core_radius_label 


imputer = SimpleImputer(strategy="median")
X_imputed = imputer.fit_transform(X)


scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)


X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

gb_regressor = GradientBoostingRegressor(n_estimators=200, learning_rate=0.05, random_state=42)
gb_regressor.fit(X_train, y_train)

GradientBoostingRegressor(learning_rate=0.05, n_estimators=200, random_state=42)

In [24]:
gb_predicted_core_radius = np.mean(gb_regressor.predict(X_test))

print(f"Predicted Core Radius of Mars (Gradient Boosting Model): {gb_predicted_core_radius:.2f} km")

model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse')


model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=1)


dl_predicted_core_radius = np.mean(model.predict(X_test))

print(f"Predicted Core Radius of Mars (Deep Learning Model): {dl_predicted_core_radius:.2f} km")

Predicted Core Radius of Mars (Gradient Boosting Model): 2389.50 km
Epoch 1/100


/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


600/600 ━━━━━━━━━━━━━━━━━━━━ 1s 653us/step - loss: 3296569.7500 - val_loss: 32442.7109
Epoch 2/100
600/600 ━━━━━━━━━━━━━━━━━━━━ 0s 682us/step - loss: 71449.9766 - val_loss: 7584.8027
Epoch 3/100
600/600 ━━━━━━━━━━━━━━━━━━━━ 0s 558us/step - loss: 49961.0664 - val_loss: 2524.4851
Epoch 4/100
600/600 ━━━━━━━━━━━━━━━━━━━━ 0s 561us/step - loss: 44665.9766 - val_loss: 668.9529
Epoch 5/100
600/600 ━━━━━━━━━━━━━━━━━━━━ 0s 565us/step - loss: 41403.7539 - val_loss: 851.3049
Epoch 6/100
600/600 ━━━━━━━━━━━━━━━━━━━━ 0s 552us/step - loss: 40778.7500 - val_loss: 585.5695
Epoch 7/100
600/600 ━━━━━━━━━━━━━━━━━━━━ 0s 564us/step - loss: 40526.9453 - val_loss: 1760.9065
Epoch 8/100
600/600 ━━━━━━━━━━━━━━━━━━━━ 0s 570us/step - loss: 40062.1562 - val_loss: 1624.9614
Epoch 9/100
600/600 ━━━━━━━━━━━━━━━━━━━━ 0s 562us/step - loss: 38866.3203 - val_loss: 373.6660
Epoch 10/100
600/600 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 39246.1445 - val_loss: 286.7428
Epoch 11/100
600/600 ━━━━━━━━━━━━━━━━━━━━ 0s 591us/step